# Load Python libraries

In [1]:
# import whatever libraries I want
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import msm_scripts as ms
import test_system_generator as tsg
import timeit

# Generate data

In [2]:
""" So what I'm doing here is generating 20 reaction coordinates and performing 50 simulations for each of coordinates.
I am currently generating 19 single well potentials (initial state chosen randomly) and 1 double well potential (initial
state at transition state)"""

simulation_data = [ ]
num_of_sims_pc = 50
num_of_coors = 20

well = ['single_well']*(num_of_coors-1)
well.append('double_well')
well = well*num_of_sims_pc

init = [0]*(num_of_coors-1)
init.append(50)
init = init*num_of_sims_pc

for i in range(len(well)):
    # define a simulation object in a given potential with a particular biasing force
    sim=tsg.simulation(potential=well[i],sim_length=5000,dt=0.01,biasing_protocol=[0,1],num_of_states=100,T=300, initial_state = init[i])
    simulation_data.append(sim.generate_data())

[0.00307098 0.00323886 0.00341574 0.00360189 0.00379758 0.00400304
 0.00421849 0.00444412 0.00468007 0.00492647 0.00518336 0.00545078
 0.00572868 0.00601698 0.0063155  0.00662402 0.00694224 0.00726978
 0.00760616 0.00795085 0.0083032  0.00866248 0.00902788 0.00939848
 0.00977328 0.01015118 0.01053102 0.01091152 0.01129134 0.01166908
 0.01204325 0.01241233 0.01277474 0.01312888 0.01347311 0.0138058
 0.0141253  0.01443001 0.01471834 0.01498877 0.01523982 0.0154701
 0.01567831 0.01586326 0.01602388 0.01615922 0.01626849 0.01635101
 0.01640631 0.01643405 0.01643405 0.01640631 0.01635101 0.01626849
 0.01615922 0.01602388 0.01586326 0.01567831 0.0154701  0.01523982
 0.01498877 0.01471834 0.01443001 0.0141253  0.0138058  0.01347311
 0.01312888 0.01277474 0.01241233 0.01204325 0.01166908 0.01129134
 0.01091152 0.01053102 0.01015118 0.00977328 0.00939848 0.00902788
 0.00866248 0.0083032  0.00795085 0.00760616 0.00726978 0.00694224
 0.00662402 0.0063155  0.00601698 0.00572868 0.00545078 0.005183

[0.00307098 0.00323886 0.00341574 0.00360189 0.00379758 0.00400304
 0.00421849 0.00444412 0.00468007 0.00492647 0.00518336 0.00545078
 0.00572868 0.00601698 0.0063155  0.00662402 0.00694224 0.00726978
 0.00760616 0.00795085 0.0083032  0.00866248 0.00902788 0.00939848
 0.00977328 0.01015118 0.01053102 0.01091152 0.01129134 0.01166908
 0.01204325 0.01241233 0.01277474 0.01312888 0.01347311 0.0138058
 0.0141253  0.01443001 0.01471834 0.01498877 0.01523982 0.0154701
 0.01567831 0.01586326 0.01602388 0.01615922 0.01626849 0.01635101
 0.01640631 0.01643405 0.01643405 0.01640631 0.01635101 0.01626849
 0.01615922 0.01602388 0.01586326 0.01567831 0.0154701  0.01523982
 0.01498877 0.01471834 0.01443001 0.0141253  0.0138058  0.01347311
 0.01312888 0.01277474 0.01241233 0.01204325 0.01166908 0.01129134
 0.01091152 0.01053102 0.01015118 0.00977328 0.00939848 0.00902788
 0.00866248 0.0083032  0.00795085 0.00760616 0.00726978 0.00694224
 0.00662402 0.0063155  0.00601698 0.00572868 0.00545078 0.005183

[0.00307098 0.00323886 0.00341574 0.00360189 0.00379758 0.00400304
 0.00421849 0.00444412 0.00468007 0.00492647 0.00518336 0.00545078
 0.00572868 0.00601698 0.0063155  0.00662402 0.00694224 0.00726978
 0.00760616 0.00795085 0.0083032  0.00866248 0.00902788 0.00939848
 0.00977328 0.01015118 0.01053102 0.01091152 0.01129134 0.01166908
 0.01204325 0.01241233 0.01277474 0.01312888 0.01347311 0.0138058
 0.0141253  0.01443001 0.01471834 0.01498877 0.01523982 0.0154701
 0.01567831 0.01586326 0.01602388 0.01615922 0.01626849 0.01635101
 0.01640631 0.01643405 0.01643405 0.01640631 0.01635101 0.01626849
 0.01615922 0.01602388 0.01586326 0.01567831 0.0154701  0.01523982
 0.01498877 0.01471834 0.01443001 0.0141253  0.0138058  0.01347311
 0.01312888 0.01277474 0.01241233 0.01204325 0.01166908 0.01129134
 0.01091152 0.01053102 0.01015118 0.00977328 0.00939848 0.00902788
 0.00866248 0.0083032  0.00795085 0.00760616 0.00726978 0.00694224
 0.00662402 0.0063155  0.00601698 0.00572868 0.00545078 0.005183

# Linear transformation of data  in to new coordinates